In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline

## 2017

In [2]:
# Load raw data
grade_2017 = 'data/grades/2017/2019-07-12T1714_Grades-SI_301_001_FA_2017.csv'
df = pd.read_csv(grade_2017)

# Create data frame
df_2017 = pd.DataFrame(index=df.index)

# Create uid based on hash of username
df_2017['uid'] = pd.Series(index=df.index, dtype='uint64')
for i in df.index:
    df_2017.loc[i, 'uid'] = hash(df['SIS Login ID'][i])
    
# Add treatment column
df_2017['treatment'] = df['Early/Late Team Work Assignment (373441)']
    
# Add midterm column
df_2017['midterm'] = df['Midterm (401592)']

In [3]:
df_2017 = df_2017[df_2017.treatment > 0]
df_2017.to_csv('preprocessed/grades_2017.csv')

## 2018

In [4]:
# Load raw data
grade_2018 = 'data/grades/2018/2019-07-12T1704_Grades-SI_301_001_FA_2018.csv'
df = pd.read_csv(grade_2018)

# Create data frame
df_2018 = pd.DataFrame(index=df.index)

# Create uid based on hash of username
df_2018['uid'] = pd.Series(index=df.index, dtype='uint64')
for i in df.index:
    df_2018.loc[i, 'uid'] = hash(df['SIS Login ID'][i])

# Add uniqename
df_2018['uniqname'] = df['SIS Login ID']
    
# Add midterm column
df_2018['midterm'] = df['Midterm (622350)']

# Use uid as index
df_2018['index'] = df_2018['uid']
df_2018 = df_2018.set_index('index')

In [5]:
# Load treatment data
assignment_file = 'data/grades/2018/Working groups and pairings/group_assignments.csv'
df = pd.read_csv(assignment_file)

# Add treatment column
df_2018['treatment'] = pd.Series(index=df_2018.index)
for i in df.index:
    uid = hash(df.loc[i, 'uniqname'])
    if df.loc[i, 'Working Group'] == 'Early':
        df_2018.loc[uid, 'treatment'] = 1
    else:
        df_2018.loc[uid, 'treatment'] = 2

In [6]:
df_2018.head()

,uid,uniqname,midterm,treatment
index,,,,
0.000000e+00,0.000000e+00,NaN,100.0,NaN
3.599830e+18,3.599830e+18,oabrams,98.0,1.0
2.410581e+18,2.410581e+18,shaelyn,97.0,1.0
-4.358879e+18,-4.358879e+18,sulayman,93.0,1.0
5.425166e+17,5.425166e+17,gbadag,83.0,2.0


In [7]:
df_2018 = df_2018[df_2018.treatment > 0]
df_2018 = df_2018.drop(columns=['uniqname'])
df_2018.to_csv('preprocessed/grades_2018.csv')